Basically, here we calculate the ernergy derivative follows eqa.6 - eqa.22 in ref https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.4.043210:
1. using pyscf to get the derivative of several function e.g. overlap, one/two-body integral, etc. in AO basis.
2. reshape the above one into the shape want we expect, e.g. (#atoms, 3, 2#MO, 2#MO) for overlap/one integral, and (#atoms, 3, 2#MO, 2#MO, 2#MO, 2#MO) for two-body integral.
3. using einsum to get the derivatives in MO basis.
4. using the derivatives in MO basis to obtain force operator.
5. using Hellmann-Feynman theorem to get energy derivative.

In [23]:
from pyscf import gto, scf, dft, lib, grad
import numpy as np
import qib
import scipy.sparse

In [3]:
#position: 0.735A
atom = '''H 0 0 0; H 0 0 0.735/lib.param.BOHR'''
#atom = '''H 0 0 0; H 0 0 0.735'''
basis = 'sto-3g'
#basis ="6-31G"
unit = 'Bohr'
#unit = 'A'
charge = 0
spin = 0
verbose = 0

mol = gto.Mole()
mol.build(atom    = atom,
          basis   = basis,
          charge  = charge,
          spin    = spin,
          unit    = unit,
          verbose = verbose)

nmo, nao, natm = mol.nao, mol.nao, mol.natm

Do RHF and obtain the AO -> MO coeff:

In [4]:
mf = scf.RHF(mol)
mf.kernel()
coeff = mf.mo_coeff
spin_coeff = np.kron(coeff, np.identity(2))

Some parameters:
$$ h_{\mu \nu} = \langle \mu | h|\nu \rangle  $$
$$ g_{\mu \nu \lambda \sigma} = \langle \mu \nu | \frac{1}{r_{12}}|\lambda \sigma \rangle  $$
$$ S_{\mu \nu} =  \langle \mu |\nu \rangle  $$
In MO basis:
$$ h_{pq} = \sum_{\mu \nu} C_{\mu p} C_{\nu q} h_{\mu \nu}    $$
$$ g_{pqrs} =  \sum_{\mu \nu} C_{\mu p} C_{\nu q} C_{\lambda r} C_{\sigma s} g_{\mu \nu \lambda \sigma}    $$
Attention that the values of $g$ may differ in different packages/papers.

In [5]:
h0 = mol.get_enuc()

h1 = np.kron(mol.get_hcore(), np.identity(2))
g = mol.intor("int2e_spinor")
S = mol.intor("int1e_ovlp")

h1_MO = np.einsum('up,vq,uv -> pq', spin_coeff, spin_coeff, h1)
g_MO = np.einsum('pqrs,pi,qj,rk,sl->ijkl', g, spin_coeff, spin_coeff, spin_coeff, spin_coeff)

Second quantization form:
$$
H = \sum_{pq} h_{pq} a_p^\dagger a_q  + \frac{1}{2} \sum_{pqrs} g_{pqrs}  a_p^\dagger a_r^\dagger a_s a_q 
$$ 
Attention the order of subscripts. We can get the correct ground energy by solving this Hamiltonian.


In [26]:
latt = qib.lattice.LayeredLattice(qib.lattice.FullyConnectedLattice((2,)), 2)
field = qib.field.Field(qib.field.ParticleType.FERMION, latt)

#attention!!! The qib.operator.MolecularHamiltonian function used here is now different from the one in github
#basically, we remove the switch transpose((0, 1, 3, 2), so that we  can define the switch by ourselves.

H = qib.operator.MolecularHamiltonian(field, h0, h1_MO, g_MO.transpose(0,2,3,1)).as_matrix().toarray()
e, v = np.linalg.eigh(H)
e_ground = e[0]
v_ground = v[:, 0]
print(e_ground)
print("the result is same as the one in qiskit")

-1.1373060357533993
the result is same as the one in qiskit


This result is the same as the one given in https://qiskit.org/documentation/nature/tutorials/01_electronic_structure.html

Attention: depending on convention, sometimes we do not have the factor $\frac{1}{2}$ in the Hamiltonian, e.g. the definition in the reveiw https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.4.043210 . In this case, we should use $g/2$, which means half of the $g$ we obtained above. Due to the reason that we will follow the equations in this paper, we will scale all the values using $g$.

To calculate the energy dirivative, we have: 
$$
\frac{dH}{dR_A} = \sum_{pq} a_p^\dagger a_q (\frac{dh_{pq}}{dR_A} - \sum_m h_{mq} \frac{dS_{mp}}{dR_A} ) + \sum_{pqrs} a_p^\dagger a_r^\dagger a_q a_s (\frac{dg_{pqrs}}{dR_A} - 2\sum_t g_{tqrs} \frac{dS_{tp}}{dR_A}  )
$$
all the terms are written in MO basis, and we will calculate them term by term. (still attention that all the $g$ and $g/2$ here is half of the values we obtained from pyscf, and we we scale related values in the calculation.)

First, the derivative of overlap matrix in MO basis: $$\frac{dS_{pq}}{dR_A} =  \sum_{\mu \nu} C_{\mu p} C_{\nu q} \frac{dS_{\mu \nu}}{dR_A}         $$
The definition of the derivative is defined in the paper https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.4.043210. Here we do not discuss the derivation and Physics intuition inside (even though it is interesting).


First we should calculate overlap matrix $S_{\mu \nu} = \langle \mu | \nu \rangle $ . What the int1e_ovlp in the pyscf give us is $S(\mu, \nu) = \langle \partial \mu | \nu \rangle $, which stands for how the overlap will change if you move the cordinate slight instead of moving the atom.

So that calculation of derivative of overlap matrix is (move the atom slightly, the change of S):

$$
\frac{dS_{\mu \nu}}{dR_A} = -\langle \partial \mu | \nu \rangle 
$$
Due to the sysmmetry of S matrix, the derivative matrix should also be symmetric:
$$
\frac{dS_{\nu \mu}}{dR_A} = \frac{dS_{\mu \nu}}{dR_A} = -\langle \partial \mu | \nu \rangle 
$$
if both of the orbitals does (or not) corresponds to the atom A, then:
$$
\frac{dS_{\mu \nu}}{dR_A} = 0
$$
Also, we should reshape the whole thing into the dimension (# of atoms, 3, # of AO orbitals, # of AO orbitals).

In [7]:
S = mol.intor("int1e_ovlp")
dS = mol.intor("int1e_ipovlp")

dS_reshape = np.zeros([natm, 3, nmo, nmo])
#what we do here is slice the matrix according to atoms, we should use mol.aoslice_by_atom()
#print(mol.aoslice_by_atom())
aoslice = mol.aoslice_by_atom()
#specify it on a atom
for i in range (natm):
        startingpoint = 0
        if i != 0:
         startingpoint = aoslice[i-1, 3]
        #get direction:
        for j in range(3):
            #get AO indices:
            #reshape:
            dS_reshape[i, j, startingpoint:aoslice[i, 3], :] = -dS[j, startingpoint:aoslice[i, 3], :]
            #dS_reshape[i, j, :, startingpoint:aoslice[i, 3]] = dS[j, :, startingpoint:aoslice[i, 3]]
            
dS_reshape += dS_reshape.swapaxes(-1, -2)
#get function of spin
dS_reshape = np.kron(dS_reshape, np.identity(2))

print(dS_reshape.shape)

(2, 3, 4, 4)


Then we can calculate the derivative of overlap matrix in MO basis: $$\frac{dS_{pq}}{dR_A} =  \sum_{\mu \nu} C_{\mu p} C_{\nu q} \frac{dS_{\mu \nu}}{dR_A}         $$

In [8]:
dS_MO = np.zeros([natm, 3, 2*nao, 2*nao])

for i in range (natm):
    for j in range (3):
        dS_MO[i, j, :, :] = np.einsum('up, vq, uv -> pq', spin_coeff, spin_coeff, dS_reshape[i, j, :, :])

print(dS_MO.shape)

(2, 3, 4, 4)


Further, we should calculate $\frac{dh_{pq}}{dR_A}$:
$$\frac{dh_{pq}}{dR_A} =  \sum_{uv} C_{\mu p} C_{\nu q} \frac{h_{\mu \nu} }{dR_A}$$ 

where $h_{pq}$ is in the MO basis and $h_{\mu \nu}$ is in the AO basis.

However, the  calculation of $\frac{dh_{\mu \nu}}{dR_A} $ is not a  trivial task. Basicly, 
$$
h = h_{kin} + h_{nuc}
$$
so that 
$$
\frac{dh}{dR_A} = \frac{dh_{kin}}{dR_A} + \frac{dh_{nuc}}{dR_A} 
$$
Calculation the first term $\frac{dh_{kin}}{dR_A}$ from pyscf is a trivial task, which  is  similar  with the  calculation  of derivation of overlap matrix before. 

But for the second term $\frac{dh_{nuc}}{dR_A} $, due to $ \langle \mu | \frac{dh_{nuc}}{dR_A} | \nu \rangle \neq 0$, we should use some tricks to calculate this term.

Calculate $\frac{dh_{kin}}{dR_A}$ first (in AO basis):


In [9]:
hkin = mol.intor("int1e_kin")
dhkin = mol.intor("int1e_ipkin")

dhkin_reshape = np.zeros([natm, 3, nmo, nmo])

for i in range (natm):
        startingpoint = 0
        if i != 0:
         startingpoint = aoslice[i-1, 3]
       
        for j in range(3):
            dhkin_reshape[i, j, startingpoint:aoslice[i, 3], :] = -dhkin[j, startingpoint:aoslice[i, 3], :]
            
dhkin_reshape += dhkin_reshape.swapaxes(-1, -2)

dhkin_reshape = np.kron(dhkin_reshape, np.identity(2))

The calculation of $\frac{dh_{nuc}}{dR_A}$ is not a trivial task. first we have:
$$
\frac{dh_{nuc}}{dR_A} (\mu, \nu) = -\langle \partial \mu|h_{nuc} | \nu \rangle  -\langle  \mu|h_{nuc} | \partial \nu \rangle + \langle  \mu|\partial h_{nuc} | \nu \rangle 
$$
We implement the first two terms:

In [10]:
#first part
hnuc = mol.intor("int1e_nuc")

dhnuc1 = mol.intor("int1e_ipnuc")
dhnuc1_reshape = np.zeros([natm, 3, nmo, nmo])

for i in range (natm):
        startingpoint = 0
        if i != 0:
         startingpoint = aoslice[i-1, 3]
        for j in range(3):
            dhnuc1_reshape[i, j, startingpoint:aoslice[i, 3], :] = -dhnuc1[j, startingpoint:aoslice[i, 3], :]
            
dhnuc1_reshape += dhnuc1_reshape.swapaxes(-1, -2)
dhnuc1_reshape = np.kron(dhnuc1_reshape, np.identity(2))

Then we calculate $\langle  \mu|\partial h_{nuc} | \nu \rangle $.

In [11]:
#second part

dhnuc2_reshape = np.zeros((natm, 3, nao, nao))
Z = mol.atom_charges()
for i in range(natm):
    with mol.with_rinv_as_nucleus(i):
        dhnuc2_reshape[i] -= Z[i] * mol.intor("int1e_iprinv")

dhnuc2_reshape += dhnuc2_reshape.swapaxes(-1, -2)
dhnuc2_reshape = np.kron(dhnuc2_reshape, np.identity(2))

dhnuc_reshape =  dhnuc1_reshape + dhnuc2_reshape 

dh1_reshape = dhkin_reshape + dhnuc_reshape

In [12]:
dh1_reshape = dhkin_reshape + dhnuc_reshape

Then use  $\frac{dh_{pq}}{dR_A} = \sum_{uv} C_{\mu p} C_{\nu q} \frac{dh_{\mu \nu}}{dR_A}  $ to calculate $\frac{dh_{pq}}{dR_A}$:

In [13]:
dh1_MO = np.zeros([natm, 3, 2*nao, 2*nao])

for i in range (natm):
    for j in range (3):
        dh1_MO[i, j, :, :] = np.einsum('up, vq, uv -> pq', spin_coeff, spin_coeff, dh1_reshape[i, j, :, :])

print(dh1_MO.shape)

(2, 3, 4, 4)


Calculation of  $\frac{dg_{pqrs}}{dR_A}$ is relatively trivial:
$$
\frac{dg_{pqrs}}{dR_A} = \sum_{\mu \nu \lambda \sigma} C_{\mu p} C_{\nu q} C_{\lambda r} C_{\sigma s} \frac{dg_{\mu \nu \lambda \sigma}}{dR_A}  
$$
First, we should get the $\frac{dg_{\mu \nu \lambda \sigma}}{dR_A}$ in correct shape (#atoms, 3, 2#AO, 2#AO, 2#AO, 2#AO):

In [14]:
dg = mol.intor("int2e_ip1")

dg_reshape = np.zeros((natm, 3, nao, nao, nao, nao))

for i in range (natm):
    startingpoint = 0
    if i != 0:
        startingpoint = aoslice[i-1, 3]
    for j in range(3):
        dg_reshape[i, j, startingpoint:aoslice[i, 3], :, :, :] = -dg [j, startingpoint:aoslice[i, 3], :, :, :]
   
dg_reshape += dg_reshape.swapaxes(-3, -4)
dg_reshape += dg_reshape.swapaxes(-1, -3).swapaxes(-2, -4)
dg_reshape.shape


(2, 3, 2, 2, 2, 2)

In [15]:
test_reshape_temp = np.zeros([natm, 3,nmo,nmo,2*nmo,2*nmo])

for i in range (natm):
    for j in range(3):
        for k in range (nmo):
            for l in range (nmo):
                test_reshape_temp[i, j, k, l, :, :] = np.kron(dg_reshape[i, j, k, l, :, :], np.identity(2))

test_reshape_temp1 = np.zeros([natm, 3,2*nmo,2*nmo,2*nmo,2*nmo])

for i in range (natm):
    for j in  range(3):
        for k in range (2*nmo):
            for l in range (2*nmo):
                test_reshape_temp1[i, j, :, :, k, l] = np.kron(test_reshape_temp[i, j, :, :, k, l], np.identity(2))



dg_reshape_spinor = test_reshape_temp1

Then calculate $\frac{dg_{pqrs}}{dR_A}  $  by:
$$
\frac{dg_{pqrs}}{dR_A} = \sum_{\mu \nu \lambda \sigma} C_{\mu p} C_{\nu q} C_{\lambda r} C_{\sigma s} \frac{dg_{\mu \nu \lambda \sigma}}{dR_A}  

$$

In [16]:
dg_MO = np.einsum('up, vq, xr, os, abuvxo ->  abpqrs', spin_coeff, spin_coeff, spin_coeff, spin_coeff, dg_reshape_spinor )
dg_MO.shape

(2, 3, 4, 4, 4, 4)

Now, we have all the terms needed. Let's have a look at the energy derivation operator:
$$
\frac{dH}{dR_A} = \sum_{pq} a_p^\dagger a_q (\frac{dh_{pq}}{dR_A} - \sum_m h_{mq} \frac{dS_{mp}}{dR_A} ) + \sum_{pqrs} a_p^\dagger a_r^\dagger a_q a_s (\frac{dg_{pqrs}}{dR_A} - 2\sum_t g_{tqrs} \frac{dS_{tp}}{dR_A}  )
$$

In [17]:
dH1_term1 = dh1_MO[1,2,:,:]
dH1_term2 = np.einsum('mq, mp -> pq', h1_MO, dS_MO[1,2,:,:])
dH1 = dH1_term1 - dH1_term2

dH2_term1 = dg_MO[1,2,:,:,:,:]
dH2_term2 = 2*np.einsum('tqrs, tp -> pqrs', g_MO, dS_MO[1,2,:,:])
dH2 = dH2_term1 - dH2_term2

dH_test = qib.operator.MolecularHamiltonian(field, 0, dH1, dH2.transpose(0,2,3,1)).as_matrix().toarray()

#convert to unit: Hatree/A
print('energy derivative alone the bond', np.vdot(v_ground, dH_test.dot(v_ground))/lib.param.BOHR, 'Hatree \ A')

energy derivative alone the bond (0.9797791720893388+0j) Hatree \ A


To verify this result, we make use of density matrices to obtain a ref value. 
First, we have 
$$
\frac{dE}{dR_A} = \sum_{pq} \gamma _{pq} \frac{dh_{pq}}{dx} + \sum_{pqrs} \Gamma _{pqrs} \frac{dg_{pqrs}}{dx} - (\sum_{pqm} \gamma _{pq} h_{mq}\frac{dS_{mp}}{dx}  +  \sum_{pqrst} \Gamma _{pqrs} g_{tqrs}\frac{S_{tp}}{dx} )
$$

We first obtain $\gamma _{pq}$

In [18]:
RDM_test = np.zeros([4,4])
test2 = np.zeros([4,4,4,4])
for i in range(4):
    for j in range(4):
        test1 = np.zeros([4,4])
        test1[i,j] = 1
        test_op = qib.operator.MolecularHamiltonian(field, 0, test1, test2).as_matrix().toarray()
        RDM_test[i,j] = (np.vdot(v_ground, test_op@v_ground))
print(RDM_test)

[[9.87559734e-01 0.00000000e+00 5.55434245e-17 0.00000000e+00]
 [0.00000000e+00 9.87559734e-01 0.00000000e+00 5.55434245e-17]
 [5.55434245e-17 0.00000000e+00 1.24402656e-02 0.00000000e+00]
 [0.00000000e+00 5.55434245e-17 0.00000000e+00 1.24402656e-02]]


/var/folders/q5/4z76qrr523xby7str3mxbjs00000gn/T/ipykernel_83046/649550118.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  RDM_test[i,j] = (np.vdot(v_ground, test_op@v_ground))


and then  $\Gamma _{pqrs}$

In [19]:
test1 = np.zeros([4,4])
RDM2_test = np.zeros([4,4,4,4])
for p in range(4):
    for q in range(4):
        for r in range(4):
            for s in range(4):
                test2 = np.zeros([4,4,4,4])
                test2[p,r,s,q] = 1

                test_op = qib.operator.MolecularHamiltonian(field, 0, test1, 2*test2).as_matrix().toarray()
                RDM2_test[p,q,r,s] = (np.vdot(v_ground, test_op@v_ground))

/var/folders/q5/4z76qrr523xby7str3mxbjs00000gn/T/ipykernel_83046/3170953354.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  RDM2_test[p,q,r,s] = (np.vdot(v_ground, test_op@v_ground))


using the density matrices to calculate the value:

In [24]:
dE1_term1 = np.sum(RDM_test*dh1_MO[1,2,:,:]) 
dE1_term2 = np.sum(RDM2_test*(dg_MO[1,2,:,:,:,:]/2)) 

dE2_term1 = np.sum(RDM_test*(np.einsum('mq, mp -> pq', h1_MO, dS_MO[1,2,:,:])))
dE2_term2 = 2*np.sum(RDM2_test*(np.einsum('tqrs, tp -> pqrs', g_MO/2, dS_MO[1,2,:,:])))

#convert to unit: Hatree/A
print('energy derivative alone the bond', (dE1_term1 + dE1_term2 - dE2_term1 - dE2_term2)/lib.param.BOHR, 'Hatree \ A')



energy derivative alone the bond (0.9797791720893391+0j) Hatree \ A


We have some test of the functions.... (in another file)